# Stage 10b — Time Series & Classification

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
PATH="/mnt/data/stage10b_project/data/synthetic_timeseries.csv" 

In [ ]:
d=pd.read_csv(PATH,parse_dates=["date"]).sort_values("date").reset_index(drop=True)
d["lag1"]=d["ret"].shift(1)
d["lag5"]=d["ret"].shift(5)
d["roll_mean5"]=d["ret"].shift(1).rolling(5).mean()
d["roll_std5"]=d["ret"].shift(1).rolling(5).std()
d["mom5"]=d["price"].pct_change(5).shift(1)
d["z20"]=((d["ret"].shift(1)-d["ret"].shift(1).rolling(20).mean())/d["ret"].shift(1).rolling(20).std())
d=d.dropna()
X=d[["lag1","lag5","roll_mean5","roll_std5","mom5","z20"]].values
y_reg=d["ret"].shift(-1).dropna()
X_reg=X[:len(y_reg)]
y_cls=(d["ret"].shift(-1)>0).astype(int).dropna()
X_cls=X[:len(y_cls)]
n=len(X_reg); split=int(n*0.75)
Xtr_r,Xte_r=X_reg[:split],X_reg[split:]
ytr_r,yte_r=y_reg.values[:split],y_reg.values[split:]
pipe_r=Pipeline([("s",StandardScaler()),("m",Ridge(alpha=1.0))])
pipe_r.fit(Xtr_r,ytr_r)
p_r=pipe_r.predict(Xte_r)
mae=mean_absolute_error(yte_r,p_r); rmse=mean_squared_error(yte_r,p_r)**0.5
print("MAE",mae); print("RMSE",rmse)
t=np.arange(len(yte_r)); plt.figure(); plt.plot(t,yte_r); plt.plot(t,p_r); plt.title("pred_vs_truth"); plt.show()

In [ ]:
n=len(X_cls); split=int(n*0.75)
Xtr_c,Xte_c=X_cls[:split],X_cls[split:]
ytr_c,yte_c=y_cls.values[:split],y_cls.values[split:]
pipe_c=Pipeline([("s",StandardScaler()),("c",LogisticRegression(max_iter=200))])
pipe_c.fit(Xtr_c,ytr_c)
p_c=pipe_c.predict(Xte_c)
acc=accuracy_score(yte_c,p_c); prec=precision_score(yte_c,p_c,zero_division=0); rec=recall_score(yte_c,p_c,zero_division=0); f1=f1_score(yte_c,p_c,zero_division=0)
print("ACC",acc,"PREC",prec,"REC",rec,"F1",f1)
cm=confusion_matrix(yte_c,p_c); import numpy as np; import matplotlib.pyplot as plt
plt.figure(); plt.imshow(cm); plt.title("confusion_matrix"); plt.show()